In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tkinter import Tk, Label, Button, StringVar, Entry, messagebox

In [2]:
df = pd.read_csv('clean_formodel.csv')
df.head()

,Team,MapName,Result,Margin,R,ACS,K,D,A,ADR,FK,FD,VS
0,KOI,Icebox,Loss,-2,0.904,174.4,14.6,16.6,7.2,118.0,1.6,3.2,NRG
1,NRG,Icebox,Win,2,1.126,204.2,16.6,14.6,6.2,131.4,3.2,1.6,KOI
2,KOI,Haven,Loss,-4,0.866,166.6,12.4,13.8,5.4,107.2,1.6,2.8,NRG
3,NRG,Haven,Win,4,1.146,188.2,13.8,12.4,7.6,121.8,2.8,1.6,KOI
4,DFM,Haven,Loss,-9,0.574,154.4,9.0,14.2,3.2,102.8,1.4,2.0,GIA


In [3]:
# Encode the 'Result' column to numerical values (0 for Loss, 1 for Win)
le = LabelEncoder()
df['Result'] = le.fit_transform(df['Result'])

In [4]:
# Select features and target variable 
features = ['R', 'ACS', 'K', 'D', 'A', 'ADR', 'FK', 'FD', 'Margin']
X = df[features]
y = df['Result']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:
# Build a neural network model
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])



In [7]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [8]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
7/7 [==============================] - 1s 33ms/step - loss: 0.7470 - accuracy: 0.5023 - val_loss: 0.6788 - val_accuracy: 0.5091
Epoch 2/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6996 - accuracy: 0.5434 - val_loss: 0.6261 - val_accuracy: 0.6182
Epoch 3/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6610 - accuracy: 0.5525 - val_loss: 0.5806 - val_accuracy: 0.7455
Epoch 4/50
7/7 [==============================] - 0s 7ms/step - loss: 0.6446 - accuracy: 0.6256 - val_loss: 0.5420 - val_accuracy: 0.8727
Epoch 5/50
7/7 [==============================] - 0s 9ms/step - loss: 0.5431 - accuracy: 0.7352 - val_loss: 0.5081 - val_accuracy: 0.8909
Epoch 6/50
7/7 [==============================] - 0s 8ms/step - loss: 0.5693 - accuracy: 0.7032 - val_loss: 0.4771 - val_accuracy: 0.8909
Epoch 7/50
7/7 [==============================] - 0s 8ms/step - loss: 0.5053 - accuracy: 0.7352 - val_loss: 0.4480 - val_accuracy: 0.9091
Epoch 8/50
7/7 [=================

In [9]:
# Evaluate the model on the test set
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

4/4 [==============================] - 0s 3ms/step


In [12]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[63  0]
 [ 4 51]]


In [10]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)

Accuracy: 0.9661016949152542
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        63
           1       1.00      0.93      0.96        55

    accuracy                           0.97       118
   macro avg       0.97      0.96      0.97       118
weighted avg       0.97      0.97      0.97       118



In [11]:
def show_prediction():
    team1_name = team1_entry.get().upper()
    team2_name = team2_entry.get().upper()

    # Check if the entered team names exist in the dataset
    if team1_name not in df['Team'].values or team2_name not in df['Team'].values:
        messagebox.showerror("Error", "One or both of the entered team names do not exist in the dataset.")
        return

    # Fetch data for the teams
    team1_data = df[df['Team'] == team1_name][features].mean().values.reshape(1, -1)
    team2_data = df[df['Team'] == team2_name][features].mean().values.reshape(1, -1)

    # Standardize the data
    team1_data_scaled = scaler.transform(team1_data)
    team2_data_scaled = scaler.transform(team2_data)

    # Predict
    team1_pred_proba = model.predict(team1_data_scaled)
    team2_pred_proba = model.predict(team2_data_scaled)

    # Ensure probabilities sum to 100%
    total_probability = team1_pred_proba[0][0] + team2_pred_proba[0][0]
    team1_percentage = (team1_pred_proba[0][0] / total_probability) * 100
    team2_percentage = (team2_pred_proba[0][0] / total_probability) * 100

    # Display results
    result_str = (
        f"{team1_name} Win Probability: {team1_percentage:.2f}%\n"
        f"{team2_name} Win Probability: {team2_percentage:.2f}%"
    )
    messagebox.showinfo("Match Prediction", result_str)


In [10]:
# Create GUI
root = Tk()
root.title("Valorant Match Prediction")

# Team 1 Entry
Label(root, text="Enter Team 1:").grid(row=0, column=0)
team1_entry = Entry(root)
team1_entry.grid(row=0, column=1)

# Team 2 Entry
Label(root, text="Enter Team 2:").grid(row=1, column=0)
team2_entry = Entry(root)
team2_entry.grid(row=1, column=1)

# Predict Button
predict_button = Button(root, text="Predict", command=show_prediction)
predict_button.grid(row=2, column=0, columnspan=2)

# Run the GUI
root.mainloop()

1/1 [==============================] - 0s 121ms/step


c:\Users\bando\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\bando\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 25ms/step


c:\Users\bando\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\bando\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 24ms/step


c:\Users\bando\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\bando\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
